Library Imports


In [12]:
import pandas as pd
from openai import OpenAI
import os
import time
from pydantic import BaseModel
import json
from collections import Counter
import datetime
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from typing import Dict, Any

GPT API setup


In [13]:
client = OpenAI(
    organization="org-Efj3WwiBs01tiD9ogyAb1vgz",
    project="proj_ItFIKb0eOHXEFM65qPMVLpHt",
    api_key="sk-proj-waDJ9nwjNNOcQa6Ol0epT3BlbkFJbwwL9qZnqZmBMVCwtvOX",
)

Prompting Utilities


In [14]:
# Write the prompts used, and the GPT output to a folder for later inspection
def write_output(
    gpt_output: Dict[str, Any],
    user_prompt: str,
    system_prompt: str,
    prompt_title: str,
    directory: str,
    input_data_dict: pd.DataFrame,
) -> None:

    try:
        # Create the subdirectory for the pompt_title if it doesn't exist
        prompt_directory = os.path.join(
            directory, prompt_title.strip()
        )  # Strip whitespace
        os.makedirs(prompt_directory, exist_ok=True)

        # Define the file paths
        output_text_file = os.path.join(prompt_directory, "output.json")
        user_prompt_text_file = os.path.join(prompt_directory, "user_prompt.txt")
        sys_prompt_text_file = os.path.join(prompt_directory, "system_prompt.txt")
        input_data_dict_file = os.path.join(prompt_directory, "dict.csv")

        # Write the output
        with open(output_text_file, "w") as f:
            f.write(gpt_output)

        # Write the prompt
        with open(user_prompt_text_file, "w") as f:
            f.write(user_prompt)

        with open(sys_prompt_text_file, "w") as f:
            f.write(system_prompt)

        input_data_dict.to_csv(
            input_data_dict_file, index=False, sep=",", lineterminator="\n"
        )

        print(f"Output and prompt saved in directory: {prompt_directory}")

    except FileNotFoundError as e:
        print(f"Error: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [15]:
# Write the number of tokens usd, the time to retrieve the output and other metadata to a CSV
def write_prompt_metadata(
    completion_tokens: int,
    prompt_tokens: int,
    total_tokens: int,
    elapsed_time: str,
    directory: str,
    prompt_title: str,
) -> None:
    current_data = pd.DataFrame(
        [
            {
                "Completion Tokens": completion_tokens,
                "Prompt Tokens": prompt_tokens,
                "Total Tokens": total_tokens,
                "Elapsed Time": elapsed_time,
                "Prompt Name": prompt_title,
            }
        ]
    )

    prompt_metadata_path = os.path.join(directory, "prompt_metadata.csv")

    # Check if the file exists to determine whether to write the header
    if os.path.exists(prompt_metadata_path):
        current_data.to_csv(prompt_metadata_path, mode="a", header=False, index=False)
    else:
        current_data.to_csv(prompt_metadata_path, mode="w", header=True, index=False)

    print("Processing complete. Results and token/time data have been saved.")

In [16]:
# Define the pompt of the GPT, providing one system prompt, a user prompt, and the expected schema for the output
def prompt_gpt(
    system_prompt: str,
    user_prompt: str,
    prompt_title: str,
    response_format: dict[str],
    output_directory: str,
    input_data_dict: pd.DataFrame,
) -> None:

    class Output(BaseModel):
        output: list[str]

    try:
        start_time = time.time()

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            max_tokens=16000,
            response_format=response_format,
        )

        completion_tokens = completion.usage.completion_tokens
        prompt_tokens = completion.usage.prompt_tokens
        total_tokens = completion.usage.total_tokens

        gpt_output = completion.choices[0].message.content

        end_time = time.time()
        elapsed_time = end_time - start_time
        # Calculate hours, minutes, seconds, and milliseconds

        minutes = int((elapsed_time % 3600) // 60)
        seconds = int(elapsed_time % 60)
        milliseconds = int((elapsed_time % 1) * 1000)

        # Format the output as mm:ss.ms
        formatted_time = f"{minutes:02}:{seconds:02}.{milliseconds:03}"

        # Write the output of the prompting
        write_output(
            gpt_output=gpt_output,
            user_prompt=user_prompt,
            system_prompt=system_prompt,
            prompt_title=prompt_title,
            directory=output_directory,
            input_data_dict=input_data_dict,
        )

        # Write the prompt metadata
        write_prompt_metadata(
            completion_tokens=completion_tokens,
            prompt_tokens=prompt_tokens,
            total_tokens=total_tokens,
            elapsed_time=formatted_time,
            directory=output_directory,
            prompt_title=prompt_title,
        )
    except Exception as e:
        # Return zeros and the error message in case of an exception
        print(str(e))
        return 0, 0, 0, f"Error: {str(e)}"

In [17]:
def create_attribute_dict(attribute, column_name: str) -> pd.DataFrame:

    attribute = pd.DataFrame(attribute)
    attribute["original_index"] = attribute.index

    # Retreive unique values
    attribute_unique = pd.DataFrame(
        attribute[column_name].unique(), columns=[column_name]
    )

    # Create an index for unique values
    attribute_unique["unique_index"] = attribute_unique.index

    # Join unique indexs to the original values, creating a dictionary of original index, value, and unique index
    attribute_dict = pd.merge(
        attribute, attribute_unique, on=column_name, how="left", suffixes=("", "_df2")
    )

    attribute_dict.rename(columns={"index_df2": "unique_index"}, inplace=True)

    return attribute_dict, attribute_unique

Load Dataset


In [18]:
dataset = "hospital"

dataset = pd.read_csv(f"./datasets/{dataset}/dirty.csv")

Deriving Context


In [19]:
# Prompt response format
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "output": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "datatype": {
                                "type": "string",
                                "description": "A description of the datatype of an attribute",
                            },
                            "range": {
                                "type": "string",
                                "description": "A description of the range of the values in an attribute, and how they inform the context",
                            },
                            "meaning": {
                                "type": "string",
                                "description": "The inherit meaning that a column has based on its values",
                            },
                        },
                        "required": ["datatype", "range", "meaning"],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["output"],
            "additionalProperties": False,
        },
    },
}

In [20]:
system_prompt_context = """You are given a few example values from a column in a relational dataset. You have to inspect the values and derive their context in the form of a description for each column. 
The values themselves may contain valuable clues to help derive the context. Check for the following clues:

1. Datatype
- The datatype of a column could help give insight into how the values might be used or interpreted. 
Example:
- Columns with decimal values rounded to two poistions could indicate that the column stores currency values.
- Columns with a specific pattern, like a date format, could indicate a date of a certain kind. Other patterns could exist with hidden meaning, like an employee number that contains the employees birthdate.
- Text values could represent different meaning, like the names of people or places, or be the nouns of objects. Such columns could also be descriptions of some kind.

2. Range of values.
- The range of values could give an indication of the context of the attribute.
Example: 
- Values that range between 2.06 and 1.5 could indicate the length of a person. So could values between -50 and +50 indicated degrees celcuis. Many other contexts may exist that is given by the range of numeric values.
- Dates between 1930 and 2024 might indicated birthdates. Dates with a smaller spread could represent transactions or deliveries.
- Text values with long strings are more likely to be descriptions than names or nouns.

3. Inherit meaning
- In some cases the values themselves might give a clear indication of their meaning.
Example:
- Text values like John, Sarah and Marcy indicates that the column is used for first names. Values like New York, London, and Dubai, Indicate that the columns are used for city names.
- Some numeric values might include unit measurement symbols, like oz or km. Combining this information with the type of value and range could inform on the context of the attribute.
- Inherit meaning is most likely to be observed in text columns.
- Some columns might have hidden meaning in only part of the values. For example, the first three letters of US social security numbers indicate the area where the code was issued, and could indicate a hidden dependecy with a column like area code or area name. Always check non natural lanuage text fields for such patterns.

Instructions:
For each set of values:
1. Describe the datatype.
2. Summarize the range of the values.
3. Identify inherit meaning if possible.
"""

In [21]:
def generate_context_attribute_string(attribute_unique) -> str:
    if attribute_unique[attribute_unique.columns[0]].dtype != "int64":
        attribute_unique[attribute_unique.columns[0]] = attribute_unique[
            attribute_unique.columns[0]
        ].apply(lambda x: x.strip())
        attribute_unique[attribute_unique.columns[0]] = attribute_unique[
            attribute_unique.columns[0]
        ].apply(lambda x: f"|{x}|")
    attribute_unique_string = attribute_unique.to_csv(
        index=False, header=False, lineterminator="\n"
    )

    return attribute_unique_string

In [22]:
# For each column in the dataset derive the context while prompting the GPT
for col in dataset.columns:

    attribute = dataset[col].copy()
    attribute_dict, attribute_unique = create_attribute_dict(attribute, col)

    directory = (
        r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context"
    )

    attribute.to_csv(index=False, header=True, lineterminator="\n", sep=",")

    user_prompt = f"""Input:
The following is a list of unique values in an attribute. 
Check the values and derive the context as indicated. The attribute values is delimited with "|value|".
{generate_context_attribute_string(attribute_unique)}
"""

    prompt_gpt(
        system_prompt_context, user_prompt, col, response_format, directory, attribute
    )

Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context\index
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context\provider_number
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context\name
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context\address_1
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context\address_2
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Do

In [23]:
directory = (
    r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\context"
)

# Initialize an empty dictionary to store data for each folder
data_dict = {}

# Loop through all subdirectories and files
for root, dirs, files in os.walk(directory):
    for file in files:
        # Get the folder name (which will be the column name)
        folder_name = os.path.basename(root)
        if file.endswith(".json"):
            # Construct the full file path

            file_path = os.path.join(root, file)

            # Open and load the JSON file
            with open(file_path, "r") as json_file:
                try:
                    json_data = json.load(json_file)

                    # Retrieve the context and join the list into a single string
                    context = "; ".join(
                        [item["meaning"] for item in json_data["output"]]
                    )
                    data_dict[folder_name] = context

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_path}: {e}")
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

# Convert the dictionary to a DataFrame
output = pd.DataFrame(list(data_dict.items()), columns=["attribute", "context"])

# Ensure the 'attribute' column in the output matches the column names in dataset
output["attribute"] = pd.Categorical(
    output["attribute"], categories=dataset.columns.str.strip(), ordered=True
)

# Sort the DataFrame according to the order in the array_of_strings
output = output.sort_values("attribute")

# Write the DataFrame to CSV without extra quotes or escape characters
output.to_csv("./output/context/context_output.csv", index=False, quoting=0)

Context Refinement


In [24]:
context = pd.read_csv("./output/context/context_output.csv")

In [25]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "output": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "attribute_index": {
                                "type": "number",
                                "description": "The index in the list where the attribute column description occurs",
                            },
                            "context": {
                                "type": "string",
                                "description": "The enriched contextual description of each attribute column",
                            },
                        },
                        "required": [
                            "attribute_index",
                            "context",
                        ],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["output"],
            "additionalProperties": False,
        },
    },
}

In [26]:
context_refinement_prompt = """You are given contextual descriptions of the columns of a relational dataset. You have to analyze these descriptions and enrhich them by taking the context of all columns into consideration.

The context of the entire table is derived with the following steps:

1. Determine dependencies between columns.
- Certain columns may have dependecies, like date of birth and age, or state code and city.
Example:
A column that is suspected to have state codes could indicated that there is a possible dependency with columns that stores city names.
Example:
The first three letters of US social security numbers indicate the area where the code was issued, and could indicate a hidden dependency with a column like area code or area name. Always check non natural lanuage text fields for such patterns.

2. View the columns as a whole. 
- When you consider all of the columns in a dataset it could give an indication on the meaning of the columns. 
Example:
If a dataset contains a column that is a positvie percentage, a negative percentage, a monetary value, and a date it is likely data on a stock portfolio. This information could be used to enrhich the context of the columns.
"""

In [27]:
def generate_context_enriched_string(context: pd.DataFrame) -> str:
    context_delimited = context.drop(context.columns[0], axis=1).reset_index()
    context_delimited[context_delimited.columns[1]] = context_delimited[
        context_delimited.columns[1]
    ].apply(lambda x: f"|{x}|")

    context_delimited[context_delimited.columns[0]] = context_delimited[
        context_delimited.columns[0]
    ].apply(lambda x: f"[{x}]")

    context_delimited_string = context_delimited.to_csv(
        index=False, header=False, lineterminator="\n", sep=">"
    )

    return context_delimited_string

In [28]:
directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\enriched_context"

user_prompt = f"""Input:
The first column of the input is an index and is delimited with "[index]". The context that has to be enriched is in the second column and delimited with "|context|".
The relationship between an index is indicated using " > ".
A index value pair is indicated by "[index]>|context|".
{generate_context_enriched_string(context)}
"""

prompt_gpt(
    context_refinement_prompt,
    user_prompt,
    "context_refinement",
    response_format,
    directory,
    context,
)

Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\enriched_context\context_refinement
Processing complete. Results and token/time data have been saved.


In [29]:
directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\enriched_context\context_refinement\output.json"


with open(directory, "r") as json_file:
    try:
        json_data = json.load(json_file)
        # Retrieve the annotation and index from the json output and create a dataframe
        context = [item["context"] for item in json_data["output"]]
        attribute_index = [item["attribute_index"] for item in json_data["output"]]
        # print(file_path)
        # print(index)
        attribute_context = pd.DataFrame(
            {"attribute_index": attribute_index, "context": context}
        )

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file {file_path}: {e}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

Attribute Level Prompting


In [30]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "output": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "explanation": {
                                "type": "string",
                                "description": "The explanation for why a value is considered an error",
                            },
                            "index": {
                                "type": "number",
                                "description": "The index in the list where the value occurs",
                            },
                            "annotation": {
                                "type": "number",
                                "description": "The annotation denoting whether a value is an error or not",
                                "enum": [1, 0],
                            },
                            "possible_repair": {"type": "string"},
                        },
                        "required": [
                            "explanation",
                            "index",
                            "annotation",
                            "possible_repair",
                        ],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["output"],
            "additionalProperties": False,
        },
    },
}

In [31]:
numeric_prompt = """You are given a list of numeric values with their corresponding index. You have to identify all errors in the list.
"""

In [32]:
text_prompt = """You are given a list of unique values in a column with their corresponding index. You have to find all syntactic errors in the dataset and recommend a possible repair.
A syntactic error occurs when a value does not conform to the structure or domain of correct values. The domain and structure of correct values have to be derived from the values themselves.
A semantic error occurs when a value falls outside of the reasonable context of a column. Use the context description to determine if a value is a semantic error.
You have to annotate an error with a '1' and a correct value of the '0' in the output.
For the possible repair only provide the reparied value is output.
You also have to provide a brief explanation referencing the examples a proof for each annotation.
Evaluate each value and provide an annotation and explanation regardless of error status.
Values denoting empty or null values can be found in any given context, and are considered correct.
Note! Only check for syntactic errors. Do not check for language errors.

Syntactic deviations can be one of the following examples
1. Invalid characters
- Characters appear in values that do not often appear in others or make them uninterpretable
Example 1
1, John = 0
2, Greg = 0
3, Frank15 = 1

Example 2 
15, Apple = 0
16, Pxar = 1
17, Banana = 0

2. Misspelling
- Words that are misspelled. Values that are considered names are less likely to be misspelt.
Example 1
2, Blue = 0
3, Green = 0
4, Orage = 1

Example 2 
67, Reservation for two people near the window = 0
68, Reservatton for five poeple at the entrance = 1
69, Three humans arriving at 9 for drinks = 0
70, A single peersonn at three = 1

3. Pattern non-conformity
- Some values may have a common pattern with certain values deviating from this pattern. There might exist more than one valid pattern in a single attribute.
Possible repairs should attempt to conform with the most prevalent patterns. Removing a pattern does not constitute a soluition.
Example 1
32, 2024/03/12 19:00 = 0
33, 2024-12-31 12:00 = 1
34, 1994/01/13 15:15 = 0
35, 12:00am 3 January 2024 = 1

Example 2
70, Admin123 = 0
71, Bob443 = 0
72, 99Alex = 1
"""

# And a context description of the attribute.
# 4. Contextual meaning
# - The contextual meaning of a column could provide some clues as to the expected values in a column.
# Example 1
# Contextual meaning: A column possibly storing state codes
# 22, MA = 0
# 23, TX = 0
# 24, ZA = 1

In [33]:
def generate_attribute_prompt_string(attribute_unique) -> str:
    attribute_delimited = attribute_unique[
        [attribute_unique.columns[1], attribute_unique.columns[0]]
    ]

    if attribute_delimited[attribute_unique.columns[0]].dtype != "int64":
        attribute_delimited.loc[:, attribute_unique.columns[0]] = attribute_delimited[
            attribute_unique.columns[0]
        ].apply(lambda x: x.strip())

    attribute_delimited.loc[:, attribute_unique.columns[0]] = attribute_delimited[
        attribute_unique.columns[0]
    ].apply(lambda x: f"|{x}|")

    attribute_delimited.loc[:, attribute_unique.columns[1]] = attribute_delimited[
        attribute_unique.columns[1]
    ].apply(lambda x: f"[{x}]")

    attribute_unique_string = attribute_delimited.to_csv(
        index=False, header=False, lineterminator="\n", sep=">"
    )

    return attribute_unique_string

In [57]:
count = 0
for col in dataset.columns:
    attribute = pd.DataFrame(dataset[col].copy())
    context = attribute_context[attribute_context["attribute_index"] == count][
        "context"
    ].values[0]
    count += 1

    attribute_dict, attribute_unique = create_attribute_dict(attribute, col)

    if attribute[attribute.columns[0]].dtype == object:
        system_prompt = text_prompt
    else:
        system_prompt = numeric_prompt

    attribute_unique.columns = ["value", "index"]
    json_sample = attribute_unique.to_json(orient="records", indent=4)

    # Context:
    # The context is delimited as "#Context#'
    # #{context}#

    user_prompt = f"""Input:
    {json_sample}
"""

    #     The first column of the input is an index and does not have to be checked for errors and is delimited with "[index]". The data to be checked is in the second column and delimited with "|value|".
    # The relationship between an index is indicated using " > ".
    # A index value pair is indicated by "[index]>|value|".
    # {generate_attribute_prompt_string(attribute_unique)}

    directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\attribute_output"

    # os.makedirs(directory, exist_ok=True)
    # file_path = os.path.join(directory, col.strip(), "dict.csv")

    # # Save the attribute dictionary to a CSV file
    # attribute_dict.to_csv(
    #     file_path, index=False, header=True, lineterminator="\n", sep=","
    # )

    prompt_gpt(
        system_prompt,
        user_prompt,
        col,
        response_format,
        directory,
        attribute_dict,
    )

In [58]:
# Initialize an empty dictionary to store data for each folder
data_dict = {}

# Loop through all subdirectories and files
for root, dirs, files in os.walk(directory):
    # Get the folder name (which will be the column name)
    folder_name = os.path.basename(root)

    # Skip the root directory itself (attribute_output)
    if root == directory:
        continue

    annotated_output = None
    dict_data = None

    for file in files:
        if file.endswith(".json"):
            # Construct the full file path
            file_path = os.path.join(root, file)

            # Open and load the JSON file
            with open(file_path, "r") as json_file:
                try:
                    json_data = json.load(json_file)
                    # Retrieve the annotation and index from the json output and create a dataframe
                    annotations = [item["annotation"] for item in json_data["output"]]
                    index = [item["index"] for item in json_data["output"]]

                    annotated_output = pd.DataFrame(
                        {"annotation": annotations, "index": index}
                    )

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_path}: {e}")
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

        elif file == "dict.csv":
            dict_file_path = os.path.join(root, file)
            try:
                dict_data = pd.read_csv(dict_file_path)
                dict_data.columns = dict_data.columns.str.strip()
            except Exception as e:
                print(f"Error reading dict.csv from file {dict_file_path}: {e}")

    if annotated_output is not None and dict_data is not None:
        annotated_output = annotated_output.drop_duplicates(
            subset="index", keep="first"
        )

        dict_out_merged = pd.merge(
            dict_data,
            annotated_output,
            left_on="unique_index",
            right_on="index",
            how="left",
        )  # Join unique indices to the original values

        dict_out_merged.fillna(0, inplace=True)
        data_dict[folder_name] = dict_out_merged["annotation"]
    else:
        print(f"Missing files for folder {folder_name}")

# Convert the dictionary to a DataFrame
output = pd.DataFrame(data_dict)
output = output[dataset.columns.str.strip()]  # Strip whitespace from columns
output = output.astype(int)
output.to_csv("./attribute_output/output.csv", index=False)

Testing


In [59]:
output = pd.read_csv("./attribute_output/output.csv")

In [60]:
error_annotation = pd.read_csv(
    "./datasets/hospital/errors.csv", header=None, index_col=None, skiprows=1
)

In [61]:
def calculate_metrics(df1, df2):
    # Flatten the dataframes to 1D arrays
    y_true = df1.values.flatten()
    y_pred = df2.values.flatten()

    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Precision
    precision = precision_score(y_true, y_pred)

    # Recall
    recall = recall_score(y_true, y_pred)

    f_score = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f_score

In [62]:
accuracy, precision, recall, f_score = calculate_metrics(output, error_annotation)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f_score}")

Accuracy: 0.9838
Precision: 0.8192534381139489
Recall: 0.6425269645608629
F1 score: 0.7202072538860104


In [40]:
def inspect_classification(error_annotation, output, input):
    error_annotation.columns = input.columns
    output.columns = input.columns

    calc = error_annotation.add(2)
    calc_out = output
    calc_out[calc_out == 0] = -1

    calc = calc.add(calc_out)

    # True positive calculation
    tp = calc == 4
    tp = input[tp]

    # False positive calculation
    fp = calc == 3
    fp = input[fp]

    # False negative calculation
    fn = calc == 2
    fn = input[fn]

    return tp, fp, fn

In [41]:
tp, fp, fn = inspect_classification(
    error_annotation=error_annotation,
    output=output,
    input=dataset,
)

In [42]:
tp.to_csv("./attribute_output/tp.csv")
fp.to_csv("./attribute_output/fp.csv")
fn.to_csv("./attribute_output/fn.csv")

In [43]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "output": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "columns": {
                                "type": "array",
                                "items": {"type": "number"},
                                "description": "An array of columns that might share a dependency.",
                            },
                            "dependency": {
                                "type": "string",
                                "description": "A description of the dependency between the identified columns.",
                            },
                        },
                        "required": ["columns", "dependency"],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["output"],
            "additionalProperties": False,
        },
    },
}

In [44]:
system_prompt = """
You are given a sample of ten records from a relational database. You have to determine which columns might have dependencies between them. 
For the output, only provide pairs of columns that might have dependencies. If multipule dependencies exist with a single column, provide more than one outputs for that column.

Dependencies can occur in two ways.

1. Semantic dependency
- The meaning of the values of one column determines the meaning of another. 
Example:
One column represents cities, and another countries. This may indicated that there is a dependency between the two columns, and that the citities column, contains cities that are present in that country.
Col 1                   , Col 2
Great Britain           , London
United States of America, Washington DC
South Africa            , Pretoria

Note! Other semantic dependecies may occur in different domains from the one mentioned in the example.

2. Pattern Dependency
- One column may have a pattern, which in part is based on the meaning of another column. 
Example:
One column represents emergency codes, the other emergency descriptions.
Col 1      , Col 2
FRE-003_a  , Forrest fire with damages above $2,000
FLD-001_z  , Floods that destroyed local buildings
ERQ-777_n  , Earthquakes that caused power outages 
HUR-008_t  , Hurricanes that was able to breach sea walls
"""

In [45]:
from tabulate import tabulate

directory = (
    r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency"
)

dataset_sample = dataset.sample(n=10)
# dataset_sample = dataset
dataset_sample.columns = range(dataset_sample.shape[1])
json_sample = dataset_sample.to_json(orient="records", indent=4)
# print(json_sample)
# attribute_unique_string = attribute_delimited.to_csv(
#         index=False, header=False, lineterminator="\n", sep=">"
#     )
# dataset_sample.to_csv(index=False, lineterminator="\n", sep=">")
# formatted_table = tabulate(
#     dataset_sample, headers="keys", tablefmt="pretty", showindex=False
# )

user_prompt = f"""Input:
The following is a formatted table with the data to be checked.
{json_sample}
"""

prompt_gpt(
    system_prompt,
    user_prompt,
    "output",
    response_format,
    directory,
    dataset_sample,
)

Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency\output
Processing complete. Results and token/time data have been saved.


In [46]:
directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency\output\output.json"


with open(directory, "r") as json_file:
    try:
        json_data = json.load(json_file)
        # Retrieve the annotation and index from the json output and create a dataframe
        columns = [item["columns"] for item in json_data["output"]]
        dependency = [item["dependency"] for item in json_data["output"]]
        # print(file_path)
        # print(index)
        dependencies = pd.DataFrame({"columns": columns, "dependency": dependency})

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file {file_path}: {e}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

In [47]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "math_response",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "output": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "explanation": {
                                "type": "string",
                                "description": "The explanation for why a value is considered an error",
                            },
                            "index": {
                                "type": "number",
                                "description": "The index in the list where the value occurs",
                            },
                            "column": {
                                "type": "number",
                                "description": "The column where the violation occured",
                            },
                            "annotation": {
                                "type": "number",
                                "description": "The annotation denoting whether a value is an error or not",
                                "enum": [1, 0],
                            },
                            "possible_repair": {"type": "string"},
                        },
                        "required": [
                            "explanation",
                            "index",
                            "annotation",
                            "possible_repair",
                            "column",
                        ],
                        "additionalProperties": False,
                    },
                },
            },
            "required": ["output"],
            "additionalProperties": False,
        },
    },
}

In [48]:
system_prompt = """
You are given data from columns in a dataset that is said to have a dependency between each other. You have to detect violations in this dependency.

You are also given a description of the dependency that you can use to help you identify violations.

For the possible repair provide only the repair value.
"""

In [49]:
def generate_dependency_prompt_string(columns: pd.DataFrame) -> str:
    # Create a true copy of the DataFrame to avoid warnings about modifying slices
    data_columns = columns.copy()

    # Remove duplicate rows based on all columns
    unique_combinations = data_columns.drop_duplicates(
        subset=data_columns.columns.tolist()
    ).copy()

    # Add index as a new column, and format it with brackets using .loc[]
    unique_combinations["index"] = (
        unique_combinations.index
    )  # Ensure you're adding a new column
    unique_combinations["index"] = unique_combinations["index"].apply(
        lambda x: f"[{x}]"
    )

    # Apply delimiters to other columns
    for col in unique_combinations.columns:
        if col != "index":  # Skip the 'index' column
            unique_combinations.loc[:, col] = unique_combinations[col].apply(
                lambda x: f"|{x}|"
            )

    # Reorder columns to move 'index' to the front
    cols = ["index"] + [col for col in unique_combinations.columns if col != "index"]
    unique_combinations = unique_combinations[cols]

    # Convert the DataFrame to CSV format with the required delimiters
    unique_combinations_string = unique_combinations.to_csv(
        index=False, header=False, lineterminator="\n", sep=","
    )

    return unique_combinations_string

In [50]:
for index, dependency in dependencies.iterrows():

    data_columns = dataset[dataset.columns[dependency.columns]]
    # unique_combinations = data_columns.drop_duplicates()
    string = generate_dependency_prompt_string(data_columns)
    directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations"

    column_string = "|".join(str(num) for num in dependency.columns)
    # column_string = "| col:".join(map(str, dependency.columns))
    # print(column_string)

    data_columns.columns = dependency.columns
    json_sample = data_columns.to_json(orient="records", indent=4)

    user_prompt = f"""Input:
The dependency identified in this table is defined as follows:
{dependency.dependency}   

The following is a formatted table with the data to be checked.
{json_sample}
"""
    #     [index],|{column_string}|
    # {string}

    prompt_gpt(
        system_prompt,
        user_prompt,
        str(index),
        response_format,
        directory,
        data_columns,
    )

Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations\0
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations\1
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations\2
Processing complete. Results and token/time data have been saved.
Output and prompt saved in directory: D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations\3
Processing complete. Results and token/time data have been saved.


In [51]:
import os
import json
import pandas as pd

# Path to the directory containing the JSON files
directory = r"D:\Documents\UU\Thesis\Artifact\CAED\dataset_analyzer\notebook\output\dependency_violations"

# Assuming the original DataFrame structure is known
original_dataframe = dataset  # Replace with your actual DataFrame

# Initialize the DataFrame with zeros
annotated_output = pd.DataFrame(
    0, index=original_dataframe.index, columns=original_dataframe.columns
)

# Loop through all subdirectories and files
for root, dirs, files in os.walk(directory):
    # Get the folder name (which will be the column name)
    folder_name = os.path.basename(root)

    # Skip the root directory itself (attribute_output)
    if root == directory:
        continue

    for file in files:
        if file.endswith(".json"):
            # Construct the full file path
            file_path = os.path.join(root, file)

            # Open and load the JSON file
            with open(file_path, "r") as json_file:
                try:
                    json_data = json.load(json_file)

                    # Retrieve the annotation, index, and column from the json output
                    annotations = [item["annotation"] for item in json_data["output"]]
                    indices = [item["index"] for item in json_data["output"]]
                    columns = [item["column"] for item in json_data["output"]]

                    # Create a temporary DataFrame to store the current annotations
                    current_annotations = pd.DataFrame(
                        {"annotation": annotations, "index": indices, "column": columns}
                    )

                    # Update the annotated output DataFrame
                    for _, row in current_annotations.iterrows():
                        index = row["index"]
                        column = row["column"]
                        annotation = row["annotation"]

                        # Ensure the index is within bounds
                        if index < len(original_dataframe):
                            # Get the column name from the index
                            col_name = original_dataframe.columns[column]

                            # Replace the original data with the annotation (if 1, else keep original)
                            if annotation == 1:  # Only replace if there's an error
                                annotated_output.at[index, col_name] = {annotation}
                            else:
                                annotated_output.at[index, col_name] = (
                                    0  # Fill with 0 if no error
                                )

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_path}: {e}")
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

# Save to CSV
annotated_output.to_csv("./dependency_violations/output.csv", index=False)

In [ ]:
tp, fp, fn = inspect_classification(
    error_annotation=error_annotation,
    output=annotated_output,
    input=dataset,
)

In [ ]:
tp.to_csv("./output/dependency_violations/tp.csv")
fp.to_csv("./output/dependency_violations/fp.csv")
fn.to_csv("./output/dependency_violations/fn.csv")

In [52]:
import pandas as pd

# Load the two datasets
dataset_1 = pd.read_csv(
    "./output/dependency_violations/dependency_output.csv"
)  # Path to first dataset
dataset_2 = pd.read_csv(
    "./output/attribute_output/output.csv"
)  # Path to second dataset

# Ensure both datasets have the same structure (columns and index)
# Initialize a consolidated DataFrame with the same shape as the original datasets
consolidated_data = pd.DataFrame(0, index=dataset_1.index, columns=dataset_1.columns)

# Loop through the indices and columns to consolidate annotations in place
for col in dataset_1.columns:
    for index in dataset_1.index:
        # Check if the current index has an error in dataset 1
        error_1 = dataset_1.at[index, col]
        error_2 = (
            dataset_2.at[index, col] if index < len(dataset_2) else 0
        )  # Handle cases where index exceeds

        # Logic to consolidate annotations
        if error_1 == 1 or error_2 == 1:  # If either dataset has an error
            consolidated_data.at[index, col] = 1  # Mark as error
        else:
            consolidated_data.at[index, col] = 0  # No error


# Save to CSV
consolidated_data.to_csv("./output/consolidated_error_annotations.csv", index=False)

In [53]:
accuracy, precision, recall, f_score = calculate_metrics(
    consolidated_data, error_annotation
)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f_score}")

Accuracy: 0.93515
Precision: 0.8722986247544204
Recall: 0.2649164677804296
F1 score: 0.4064073226544622


In [54]:
tp, fp, fn = inspect_classification(
    error_annotation=error_annotation,
    output=output,
    input=dataset,
)

In [55]:
tp.to_csv("./output/tp.csv")
fp.to_csv("./output/fp.csv")
fn.to_csv("./output/fn.csv")